# install requirements

In [11]:
!pip install kss

     |████████████████████████████████| 42.3 MB 77 kB/s 
     |████████████████████████████████| 184 kB 48.4 MB/s 
  Created wheel for kss: filename=kss-3.1.0.4-py3-none-any.whl size=42336591 sha256=f0431199dc06768db3948294e184f27a39868a3fac80923b75224c49db1d0e0a
  Stored in directory: /root/.cache/pip/wheels/94/d8/3c/b5f02f814e08c3e2f35e32ae2ac92a34c8412ed6f92ff470ce
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186469 sha256=a9cf33b96a259aa4f1442dbd7db33053332b4b50a265c758c52603945599eff0
  Stored in directory: /root/.cache/pip/wheels/e4/61/e7/2fc1ac8f306848fc66c6c013ab511f0a39ef4b1825b11363b2
Successfully built kss emoji


In [20]:
!pip install transformers==3.3.0

     |████████████████████████████████| 1.1 MB 8.3 MB/s 
     |████████████████████████████████| 1.2 MB 50.4 MB/s 
     |████████████████████████████████| 3.0 MB 50.3 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.9.2
    Uninstalling transformers-4.9.2:
      Successfully uninstalled transformers-4.9.2


In [23]:
!pip install mxnet gluonnlp

     |████████████████████████████████| 344 kB 8.8 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=c4d191656126c45c52ac048dfffd9f2c90d9eec47dc75ffeee540f4924ee011c
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp


# load kobert model - pretrained

In [15]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-lji4w2ia
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-lji4w2ia
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=8df90e41d747617074770c0102e36fd613cae8df70027d47d62bb895efcfb827
  Stored in directory: /tmp/pip-ephem-wheel-cache-gk1smunp/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [2]:
cd /content/drive/MyDrive/bert

/content/drive/.shortcut-targets-by-id/1nWheKWdEjM291jMl9V_KokMTeNb7xTjM/bert


In [45]:
import torch
import re
import kss
import pandas as pd
import gluonnlp as nlp
import numpy as np

from torch.utils.data import Dataset, DataLoader
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [25]:
device = torch.device("cuda:0")

In [26]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [49]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [53]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load('nlp_checkpoint.pt'))
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

# predict custom fairytale text data

In [54]:
def cleaning(sent):
  sent_clean = re.sub("[\t\n]", "", sent)
  sent_clean = re.sub("[^가-힣ㄱ-하-ㅣ!.,?\\d\\s]", "", sent_clean)
  sent_clean = re.sub("\s{2,}", " ", sent_clean)
  return sent_clean

In [55]:
with open("redhat_text.txt", "r") as f:  ## you can load your own fairytale text data
    data = f.read()

In [56]:
data = cleaning(data)
data

'옛날 옛날에 모두의 사랑을 받는 작고 귀여운 소녀가 있었습니다. 하지만 그 소녀를 가장 사랑하는 것은 그녀의 할머니였습니다. 할머니는 소녀에게 무엇을 줘야 할지 몰랐습니다. 한번은 할머니가 소녀에게 붉은 벨벳으로 만들어진 모자를 선물했습니다. 소녀에게 그 모자가 잘 어울렸고, 소녀가 그 모자가 아닌 다른 것은 쓰지 않으려고 했습니다. 그래서 그 소녀는 빨간 모자라고 불렸습니다. 어느 날, 소녀의 엄마가 그녀에게 말했습니다. 빨간 모자, 여기로 와보렴. 여기 케이크 한 조각과 와인 한 병을 할머니에게 가져다 주렴. 할머니가 편찮으시니까 네가 가면 기뻐하실 거야. 더워지기 전에 출발하렴. 그리고 할머니 댁에 갈 때, 조심해서 가고 길에서 벗어나지 마렴. 그렇지 않으면 네가 넘어져서 병을 깨뜨릴 거야. 그러면 할머니는 아무것도 받지 못하신단다. 그리고 할머니 방에 가면, 먼저 인사하고 방 안 구석구석을 살펴보는 것을 잊지 말아라!엄마 말 잘 들을게요. 빨간 머리가 엄마에게 말했습니다. 하지만 할머니는 마을에서 30분 떨어진 거리에 있는 숲에 사셨습니다. 빨간 모자가 숲에 같을 때, 그녀는 늑대를 만났습니다. 하지만 빨간 모자는 늑대가 얼마나 나쁜 동물인지 몰랐고 늑대를 무서워하지 않았습니다. 안녕, 빨간 모자! 늑대가 말했습니다. 고마워, 늑대! 빨간 모자야, 이렇게 일찍 어딜 가니? 할머니께 무엇을 가져가니? 케이크와 와인. 어제 우리가 편찮으신 할머니께 좋은 것을 드리고 기운 내게 해드리려고 케이크를 만들었어. 빨간 모자야, 너희 할머니는 어디 사시니? 여기서도 15분 더 숲 속으로 가야 해. 3개의 오크 나무 아래에 할머니 집이 있어. 그 아래에는 개암나무가 있어. 너도 거기를 알거야. 빨간 모자가 말했습니다. 늑대는 혼자 생각했습니다. 저 어리고 상냥한 한입거리가 노인보다 훨씬 맛있을거야. 둘 다 잡아먹으려면 빨리 꾀를 생각해내야 해. 늑대가 빨간 모자 옆으로 가서 말했습니다. 빨간 모자야, 여기 주변에 있는 예쁜 꽃들 좀 봐. 왜 주변을 둘러보지 않니? 

In [57]:
new_datalines = kss.split_sentences(data)
len(new_datalines)

118

In [58]:
pred = pd.DataFrame({"contents": new_datalines})

In [59]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [60]:
## Setting parameters
max_len = 39
batch_size = 32

In [61]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [62]:
def predict(predict_sentence):
    sentiment_list = []

    for sentence in predict_sentence:
        data = [sentence, '0']
        dataset_another = [data]

        another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
        test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=4)
        
        model.eval()
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)
            test_eval=[]

            for i in out:
                logits=i
                logits = logits.detach().cpu().numpy()
                test_eval.append(np.argmax(logits))

            sentiment_list.append(test_eval[0])

    return sentiment_list

In [63]:
sents = list(pred['contents'])
sentiment_list = predict(sents)

In [64]:
pred['sentiment'] = sentiment_list
pred.head()

,contents,sentiment
0,옛날 옛날에 모두의 사랑을 받는 작고 귀여운 소녀가 있었습니다. 하지만 그 소녀를 ...,1
1,할머니는 소녀에게 무엇을 줘야 할지 몰랐습니다.,0
2,한번은 할머니가 소녀에게 붉은 벨벳으로 만들어진 모자를 선물했습니다.,1
3,"소녀에게 그 모자가 잘 어울렸고, 소녀가 그 모자가 아닌 다른 것은 쓰지 않으려고 ...",1
4,그래서 그 소녀는 빨간 모자라고 불렸습니다.,0


In [65]:
# Save csv
pred.to_csv('0827_redhat_onlytales.csv', encoding='utf-8-sig', index=True)

# final output for Tacotron2

In [66]:
prefix_ref = "/content/drive/MyDrive/emotion-tts/jvoice/jvoice_3_L.wav"  ## get your own voice reference wav directory
speaker_id = "pjh" ## your speaker_id
tts_text = ""

for index, row in pred.iterrows():
  tts_text += prefix_ref + "|" + row['contents'] + "|"  + speaker_id + "|" + str(row['sentiment']) + "|" + str(row['sentiment'])
  tts_text += "\n"

In [67]:
with open('bert_to_tts_redhat.txt', 'w', encoding='utf-8')as f:
  f.write(tts_text)